In [ ]:
# ─── analyse_distribution.ipynb — cellule unique ───────────────────────────────
import openpmd_api as io
import numpy as np
import matplotlib.pyplot as plt

# ─── 1) Paramètres ─────────────────────────────────────────────────────────────
#FILE      = "3D_Particles_Gauss3D/electrons.bp5"   # chemin vers votre fichier OpenPMD
#FILE      = "3D_dataset/openpmd_000100.bp5"   # chemin vers votre fichier OpenPMD
FILE      = "data_gab2/openpmd_50000.bp/"   # chemin vers votre fichier OpenPMD
SPECIES   = "electrons_inj"                    # nom de l'espèce
ITERATION = 50000                             # itération à lire (0 pour la première)
BINS      = 100                            # nombre de bins pour l’histogramme
# ────────────────────────────────────────────────────────────────────────────────

MEV_C_CONVERSION = 5.3442859e-22

# 2) Ouvre la série en lecture seule
series = io.Series(FILE, io.Access.read_only)

# 3) Sélectionne l’itération
it = series.iterations[ITERATION]

# 4) Récupère l’espèce et ses composantes px, py, pz
sp = it.particles[SPECIES]
px = sp["momentum"]["x"][:]
py = sp["momentum"]["y"][:] 
pz = sp["momentum"]["z"][:] 
weights = sp["weighting"][:]

# 5) Ferme la série
series.close()

# 6) Calcule le module p = sqrt(px² + py² + pz²)
px = px / MEV_C_CONVERSION
py = py / MEV_C_CONVERSION
pz = pz / MEV_C_CONVERSION
p_mod = np.sqrt(px**2 + py**2 + pz**2) 

# 7) Calcule histogramme via numpy.histogram
counts, bin_edges = np.histogram(p_mod, bins=BINS)

# 8) Trace l’histogramme
plt.figure(figsize=(8,5))
plt.bar(
    (bin_edges[:-1] + bin_edges[1:]) / 2,  # abscisses = centres de bins
    counts,
    width=(bin_edges[1] - bin_edges[0]),
    edgecolor='black'
)
plt.xlabel("Module de l'impulsion $p$ (MeV/c)")
plt.ylabel("Nombre de particules")
plt.title(f"Distribution des impulsions ({SPECIES}, it={ITERATION})")
plt.grid(alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

# 9) Somme des bins = nombre total de particules
total_via_bins = counts.sum()
total_direct     = p_mod.size
print(f"Nombre total calculé par somme des bins : {total_via_bins}")
print(f"Nombre total direct (len)             : {total_direct}")
